<a href="https://colab.research.google.com/github/antonium85/crypto-trading-analysis/blob/main/ichimoku_stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yahoo_fin

In [ ]:
pip install ta

In [ ]:
import pandas as pd
import ta
from yahoo_fin.stock_info import get_data
from datetime import datetime, timedelta

end_date = datetime.today() - timedelta(1)
start_date = end_date - timedelta(365*10)
#print("End date = ", end_date.strftime('%d-%m-%Y'))
#print("Start date = ", start_date.strftime('%d-%m-%Y'))

# replace ^FCHI (CAC40) by what you want AMZN, GOOG, META ...
df = get_data("^FCHI", start_date=start_date.strftime('%d/%m/%Y'), end_date=end_date.strftime('%d/%m/%Y'), index_as_date = True, interval="1d")

del(df['adjclose'])
df['close'] = pd.to_numeric(df['close'])
df['high'] = pd.to_numeric(df['high'])
df['low'] = pd.to_numeric(df['low'])
df['open'] = pd.to_numeric(df['open'])

df['TENKAN'] = ta.trend.ichimoku_conversion_line(df['high'],df['low'])
df['KIJUN'] = ta.trend.ichimoku_base_line(df['high'],df['low'])
df['SSA'] = ta.trend.ichimoku_a(df['high'],df['low']).shift(periods=26)
df['SSB'] = ta.trend.ichimoku_b(df['high'],df['low']).shift(periods=26)

df

In [ ]:
import matplotlib
print('Last price : ',df['close'].iloc[-1])
print('SSA[-1] = ',df['SSA'].iloc[-1])
print('SSB[-1] = ',df['SSB'].iloc[-1])

#df[['close','SSA','SSB']].iloc[-600:].plot(figsize=(20,10))
tmp = df[['close','SSA','SSB','TENKAN','KIJUN']].tail(200)
a1 = tmp.plot(figsize=(20,10))
a1.fill_between(tmp.index, tmp.SSA, tmp.SSB,color='lightgrey')

In [ ]:
def calcul_value(a,b,c,reverse=False):
  #n_value = c + b - a
  #v_value = b + b - c
  #e_value = b + b - a
  #nt_value = c + c - a
  values = c+b-a,2*b-c,2*b-a,2*c-a

  return sorted(values,reverse=reverse)

side = ''

dtrend = pd.DataFrame(columns = ['date','trend','peak'])
for index, row in df.iterrows():
  if row['close'] > row['SSA'] and row['close'] > row['SSB'] and side != 'buy' :#and row['STOCH_RSI'] < 0.8 :
    side = 'buy'
    
    myrow = pd.DataFrame({'date':index,'trend':'up','peak':0},index=[0])
    dtrend = pd.concat([dtrend,myrow])
  elif row['close'] < row['SSA'] and row['close'] < row['SSB'] and side != 'sell' :#and row['STOCH_RSI'] > 0.2 :
    side = 'sell'
    peak = 100000

    myrow = pd.DataFrame({'date':index,'trend':'down','peak':100000},index=[0])
    dtrend = pd.concat([dtrend,myrow])

  if side == 'buy'and dtrend['peak'].iloc[-1] < row['high'] :
    dtrend.iloc[-1, dtrend.columns.get_loc('peak')] = row['high']
    dtrend.iloc[-1, dtrend.columns.get_loc('date')] = index
  elif side == 'sell' and dtrend['peak'].iloc[-1] > row['low'] :
    dtrend.iloc[-1, dtrend.columns.get_loc('peak')] = row['low']
    dtrend.iloc[-1, dtrend.columns.get_loc('date')] = index

dtrend = dtrend.set_index(dtrend['date'])
dtrend.index = pd.to_datetime(dtrend.index)
del(dtrend['date'])

#print(dtrend)

value = calcul_value(dtrend['peak'].iloc[-4],dtrend['peak'].iloc[-3],dtrend['peak'].iloc[-2])
next_side = ''
target = df['close'].iloc[-1]
stop_loss = df['close'].iloc[-1]

if dtrend['trend'].iloc[-2] == 'up':
  next_side = 'sell'
  result = sorted(value,reverse=True)
  value = tuple(result)

  for v in value:
    if target < v:
      continue
    else:
      target = v
      break

  stop_loss = max(df['SSA'].iloc[-1],df['SSB'].iloc[-1])

elif dtrend['trend'].iloc[-2] == 'down':
  next_side = 'buy'
  result = sorted(value)
  value = tuple(result)

  for v in value:
    if target > v:
      continue
    else:
      target = v
      break

  stop_loss = min(df['SSA'].iloc[-1],df['SSB'].iloc[-1])

print(next_side)
print(value)
print('target : ', target)
print('stop loss : ', stop_loss)